In [ ]:
!pip install datasets transformers torch wandb

In [ ]:
from datasets import load_dataset, load_metric
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, set_seed
import numpy as np
from torch.utils.data import DataLoader
import pickle


import wandb
wandb.login()
%env WANDB_PROJECT=subset_active_learning

wandb: Currently logged in as: garylai. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=subset_active_learning


In [ ]:
class ActiveLearner():
    def __init__(self, config):
        ############ set up data ############
        set_seed(42)
        self.sst2 = load_dataset("sst")
        self.valid_ds = self.preprocess(self.sst2["validation"])
        self.test_ds = self.preprocess(self.sst2["test"])
        ############ helper functions ############
        self.tokenizer = AutoTokenizer.from_pretrained(config.model_name)
        self.metric = load_metric("accuracy")
        
    def preprocess(self, data):
        data = data.rename_column('label', 'scalar_label')
        data = data.map(lambda x: {'label' : 0 if x['scalar_label'] < 0.5 else 1})

        def tokenize_func(examples): 
            tokenized = self.tokenizer(
                examples["sentence"], padding="max_length", max_length=self.config.max_length, truncation=True
            )
            tokenized["labels"] = examples["label"]
            return tokenized

        ds = data.map(
            tokenize_func,
            remove_columns=data.column_names,
            batched=True,
        )
        ds.set_format(type="torch")
        return ds

    def compute_metrics(self, eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return self.metric.compute(predictions=predictions, references=labels)
    
    def sample_data(self, strategy, n_samples):
        if strategy == "random_sampling":
            print(f"config strategy is {config.strategy}")
            selected_indices = np.random.choice(len(self.sst2["train"]), replace=False, size=config.sampling_size)
        return selected_indices

    def train(self):
        for n_samples in self.config.sampling_sizes:
            self.step(n_samples)

    def step(self, n_samples):
        """Take an active learning step"""
        ########### set up data ######### 
        # sample new data
        sampled_data = self.sample_data(self.config.strategy, n_samples)
        # concatenate the sampled data with the original data
        self.train_data_indices.extend(sampled_data)
        train_data = self.sst2["train"].select(self.train_data_indices)
        debug_data = self.sst2["train"].select(self.train_data_indices[:8])

        self.train_ds = self.preprocess(train_data)
        self.valid_ds = self.preprocess(self.sst2["validation"])
        self.debug_ds = self.preprocess(debug_data)

        ########### set up training #########
        training_args = TrainingArguments(output_dir=dir, 
                                        max_steps=self.config.max_steps if not self.config.debug else 640, 
                                        evaluation_strategy="steps", 
                                        eval_steps=self.config.max_steps//20, 
                                        report_to="wandb", 
                                        run_name=f"{self.config.strategy}-size-{n_samples}")
        print(f"training_args: {training_args}")
        model = AutoModelForSequenceClassification.from_pretrained(self.config.model_name, num_labels=2)
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=self.train_ds if not self.config.debug else self.debug_ds,
            eval_dataset=self.valid_ds if not self.config.debug else self.debug_ds,
            compute_metrics=self.compute_metrics,
        )
        ######### train #######
        trainer.train()
        wandb.finish()

        ######## test ########
        outputs = trainer.predict(self.test_ds)
        with open(f"{dir}/test_set_evaluation.pkl", "wb") as f: 
            pickle.dump(outputs, f)

In [ ]:
@dataclass(frozen=True)
class Config:
    max_length: int = 66
    debug: bool = False
    model_name: str = "google/electra-small-discriminator"
    strategy: str = "random_sampling"
    sampling_sizes: tuple = (1000, 2000, 3000, 4000)
    max_steps: int = 10000

config = Config(debug=True)

In [ ]:
active_learner = ActiveLearner(config)
active_learner.train()

In [ ]:
for sampling_size in (3000, 5000, 7000):
  config = Config(max_length=66, debug=False, strategy="random_sampling", sampling_size=sampling_size, max_steps=10000)
  dir = f"/content/drive/MyDrive/active_learning/{config.strategy}/size_{config.sampling_size}"
  !mkdir $dir
  train(config, dir)

mkdir: cannot create directory ‘/content/drive/MyDrive/active_learning/random_sampling/size_3000’: File exists


No config specified, defaulting to: sst/default
Reusing dataset sst (/root/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

config strategy is random_sampling


  0%|          | 0/3000 [00:00<?, ?ex/s]

...

loading configuration file https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ca13c16218c6780ec76753d3afa19fcb7cc759e3f63ee87e441562d374762b3d.3dd1921e571dfa18c0bdaa17b9b38f111097812281989b1cb22263738e66ef73
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_v

Step,Training Loss,Validation Loss,Accuracy
500,0.575500,0.435988,0.816530
1000,0.453800,0.431505,0.824705
1500,0.393900,0.615340,0.811989
2000,0.366500,0.589761,0.820163
2500,0.321200,0.577386,0.828338
3000,0.256700,0.665141,0.821072
3500,0.255900,0.809920,0.816530
4000,0.178800,0.753470,0.843778
4500,0.184300,0.864364,0.822888
5000,0.141700,0.833706,0.841054


***** Running Evaluation *****
  Num examples = 1101
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/active_learning/random_sampling/size_7000/checkpoint-500
Configuration saved in /content/drive/MyDrive/active_learning/random_sampling/size_7000/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/active_learning/random_sampling/size_7000/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1101
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/active_learning/random_sampling/size_7000/checkpoint-1000
Configuration saved in /content/drive/MyDrive/active_learning/random_sampling/size_7000/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/active_learning/random_sampling/size_7000/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1101
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/active_learning/random_sampling/size_7000/checkpoin

eval/accuracy,▂▄▁▃▅▃▂█▃▇▅▆▃▄▅▆▆▆▇▆
eval/loss,▁▁▃▃▂▃▅▄▅▅▆▆▇▆█▇▇███
eval/runtime,▄▆▆▂▅▃▄▁▆▃▆█▆▆▃▅▅▄▅▃
eval/samples_per_second,▅▃▃▇▄▅▅█▃▆▃▁▃▃▅▄▄▅▄▆
eval/steps_per_second,▅▃▃▇▄▅▅█▃▆▃▁▃▃▅▄▄▅▄▆
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
train/loss,█▆▆▅▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


***** Running Prediction *****
  Num examples = 2210
  Batch size = 8


In [ ]:
# with open(f"{dir}/test_set_evaluation.pkl", "rb") as f:
#     loaded_outputs = pickle.load(f)